# 5.1二维卷积层

## 5.1.1二维互相关运算

In [1]:
import torch 
import torch.nn as nn

def corr2d(X, K):
    '''
    @X：输入数组
    @K：核数组
    '''
    H, W = X.shape
    h, w = K.shape
    Y = torch.zeros(H - h + 1, W - w + 1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum()
    return Y

In [2]:
X = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = torch.tensor([[0, 1], [2, 3]])
Y = corr2d(X, K)
print(Y)

tensor([[19., 25.],
        [37., 43.]])


## 5.1.2二维卷积层

In [3]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        '''
        @kernel_size：二维元组
        '''
        super(Conv2D, self).__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.randn(1)) # 长度为1的Tensor
        # Parameter本身是Tensor的子类，自动为参数附上梯度
        # 对nn.Module的子类，会维护一个参数的集合，会自动把Parameter注册到这个集合之中

    def forward(self, x):
        '''
        @x：卷积层的输入
        '''
        return corr2d(x, self.weight) + self.bias

## 5.1.3图像中物体边缘检测

In [4]:
X = torch.ones(6, 8)
Y = torch.zeros(6, 7)
X[:, 2: 6] = 0
Y[:, 1] = 1
Y[:, 5] = -1
print(X)
print(Y)

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])
tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])


## 5.1.4通过数据学习核数组

In [5]:
conv2d = Conv2D(kernel_size=(1, 2)) # 关注输入同一行相邻元素的变化，子区域
step = 30
lr = 0.01
for i in range(step):
    Y_hat = conv2d(X)
    l = ((Y_hat - Y) ** 2).sum()
    l.backward()
    # 梯度下降
    conv2d.weight.data -= lr * conv2d.weight.grad
    conv2d.bias.data -= lr * conv2d.bias.grad
    
    # 梯度清零
    conv2d.weight.grad.zero_()
    conv2d.bias.grad.zero_()
    if (i + 1) % 5 == 0:
        print('Step %d, loss %.3f' % (i + 1, l.item()))
        
print(conv2d.weight.data)
print(conv2d.bias.data)

Step 5, loss 11.831
Step 10, loss 2.746
Step 15, loss 0.704
Step 20, loss 0.189
Step 25, loss 0.052
Step 30, loss 0.014
tensor([[ 0.9707, -0.9685]])
tensor([-0.0012])
